In [1]:
from haystack import *
from haystack.document_stores import FAISSDocumentStore
from haystack.schema import Document
from haystack.nodes import DensePassageRetriever, FARMReader
from haystack.pipelines import ExtractiveQAPipeline
from haystack.utils import print_answers

C:\Users\nishu\Anaconda3\envs\haystack\lib\site-packages\ray\autoscaler\_private\cli_logger.py:57: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
  warnings.warn(


In [2]:
from bs4 import BeautifulSoup
import urllib.request as urllib2
from googlesearch import search
from bs4.element import Comment
import urllib.request

In [4]:
def create_documents(urls):
    
    documents = []

    for j, url in enumerate(urls):        
        try:
            hdr = {'User-Agent': 'Mozilla/5.0'}
            req = urllib2.Request(url, headers = hdr)
            page = urllib2.urlopen(req)
            soup = BeautifulSoup(page, "html.parser")
            
            for i, paragraph in enumerate(soup.findAll("p")):
                p = paragraph.text.split()
                if len(p) == 0:
                    continue
                documents.append(Document(content = " ".join(p)))   
        except:
            "crap"
        
    return documents

def create_docs(directory):
    import os
    
    docs = []

    document_names = os.listdir(f"./{directory}")
    for doc in document_names:
        text = open(f"{directory}/{doc}",  encoding = 'utf8').read()
        docs.append(Document(content = text))
        
    return docs

In [6]:
query = "What is the history of machine learning?"
urls = [url for url in search(query, tld = "co.in", num = 50, stop = 40, pause = 0.5)]   
urls = [url for url in urls if "html" not in url and "wikipedia" not in url]

In [7]:
docs = create_documents(urls)

In [8]:
document_store = FAISSDocumentStore(vector_dim=128, faiss_index_factory_str="Flat")
document_store.delete_documents()
document_store.write_documents(docs)

In [ ]:
from haystack.retriever.dense import EmbeddingRetriever

retriever = EmbeddingRetriever(document_store=document_store,
                               embedding_model="yjernite/retribert-base-uncased",
                               model_format="retribert")

document_store.update_embeddings(retriever)

Some weights of RetriBertModel were not initialized from the model checkpoint at yjernite/retribert-base-uncased and are newly initialized: ['bert_query.embeddings.position_ids']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Updating Embedding:   0%|                                                                  | 0/1617 [00:00<?, ? docs/s]

Creating Embeddings:   0%|          | 0/51 [00:00<?, ? Batches/s]

In [ ]:
from haystack.generator.transformers import Seq2SeqGenerator
from haystack.pipeline import GenerativeQAPipeline

generator = Seq2SeqGenerator(model_name_or_path="yjernite/bart_eli5", min_length = 100)
pipe = GenerativeQAPipeline(generator, retriever)

In [ ]:
ret = pipe.run(query = query)

In [ ]:
ret["answers"]